# OCR QA Score Pipeline with Impresso Package

### What is this notebook about?

This notebook demonstrates the functionality and use cases of the Impresso subpackage `ocrqa`.

At its core, this package/pipeline takes a text input and calculates a QA score using a Bloom filter. It first automatically detects the language of the text using the Impresso subpackage `langident` (unless explicitly specified). The pipeline dynamically checks all available languages based on existing Bloom filters and returns an "unsupported language" message if the language is not yet available. If supported, it retrieves the latest Bloom filter for the detected language from the Impresso Hugging Face repository: `impresso-project/OCR-quality-assessment-unigram`, which is then used to calculate the QA score.

*  QA Score: A value between 0 and 1, representing the ratio of known to unknown words in the text compared to the Bloom filter.

Additionally, as will be shown below, the pipeline provides optional arguments that allow for more advanced usage of the package.



### Prerequisites

First, you should install Impresso package:

In [1]:
!pip install impresso_pipelines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.3/505.3 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.6/321.6 kB 15.3 MB/s eta 0:00:00
  Created wheel for pybloomfiltermmap3: filename=pybloomfiltermmap3-0.6.0-cp311-cp311-linux_x86_64.whl size=430655 sha256=4f95052d892e38a10857ceb77940fcbef1d0fc2c93eed70b8dd2f93aeb47c14b
  Stored in directory: /root/.cache/pip/wheels/13/d3/6e/7dd6b4a7c9d40160ab13b243b85162c2ba42503d9f3efc6afa
Successfully built pybloomfiltermmap3


---
### Basic Usage

Unless explicitly specified, the pipeline uses the Impresso subpackage `langident` to detect the language of the text with the latest available model. For more details on the langident subpackage, please refer to the Impresso demo notebook *langident_pipeline_demo.ipynb*.

Once the language is detected, the pipeline checks if a corresponding Bloom filter exists. If available, it retrieves and uses the latest version.

In [16]:
# Start by importing the necessary module from Impresso package
from impresso_pipelines.ocrqa import OCRQAPipeline
ocrqa_pipeline = OCRQAPipeline()

Once you initialize the pipeline, you can simply provide the text you'd like to classify. This example demonstrates the use of German text.

In [17]:
de_text = "Ein kleiner Hund namens Max lebte in einem ruhigen Dorf. Jeden Tag rannte er durch die Straßen und spielte mit den Kindern. Eines Tages fand er einen geheimen Garten, den niemand kannte. Max entschied sich, den Garten zu erkunden und entdeckte viele schöne Blumen und Tiere. Von diesem Tag an besuchte er den Garten jeden Nachmittag."

In [18]:
ocrqa_pipeline(de_text)

{'language': 'de', 'score': 1.0}

The default output of the pipeline is a dictionary containing the detected language and the corresponding QA score. The QA score is rounded to one decimal place to account for minor variations, such as the presence of unusual names, which should not significantly impact the overall score.

In this example, the score is 1.0, indicating that almost all words in the text exist in the Bloom filter. This suggests that the OCR process was highly successful.

---
### Advanced Usage

This pipeline offers several additional attributes that can be used when calling it to gain a deeper understanding of the results. These attributes include `language`, `version`, `diagnostics`, `model_id`, and `supported_languages`:

*   `language`: Accepts language abbreviation strings such as "en" (English) or "de" (German). If provided, the pipeline assumes the specified language and skips the language detection step, directly using the corresponding Bloom filter.

*   `version`: Accepts a specific Bloom filter model version in the format "1.0.5" or "1.0.6". If specified, the pipeline uses the requested version (if available) and skips the automatic retrieval of the latest model.

*   `diagnostics`: Boolean. If set to True, the pipeline returns additional information, such as known_tokens, unknown_tokens, and the Bloom filter name used. For more details, see the sections below.

*   `model_id`: Boolean. If set to True, the pipeline includes the name of the Bloom filter model used in the output.

*   `supported_languages`: Boolean. If set to True, the pipeline returns a list of supported languages (i.e., languages for which a Bloom filter is available).

These attributes can be used individually, in combination with each other, or all at once, depending on the level of detail needed.

**Example 1**: `language`

In [19]:
# Using the same German text example as before
ocrqa_pipeline(de_text, language="lb")

{'language': 'lb', 'score': 0.9}

Even though the provided text is clearly in German, specifying the language as Luxembourgish, for example, forces the pipeline to use the corresponding Bloom filter for that language. If the selected language is unsupported, the pipeline will return an appropriate error message.

**Example 2**: `version`

In [20]:
# Using the same German text example as before
ocrqa_pipeline(de_text, version="1.0.5")

{'language': 'de', 'score': 1.0}

In the example above, by explicitly setting the `version` to *1.0.5* , you are instructing the pipeline to use the Bloom filter corresponding to this version, even if a more recent version is available.

**Example 3**: `diagnostics`

In [21]:
# Using the same German text example as before
ocrqa_pipeline(de_text, diagnostics=True)

{'language': 'de',
 'score': 1.0,
 'diagnostics': {'known_tokens': ['von',
   'jeden',
   'diesem',
   'tages',
   'max',
   'ein',
   'fand',
   'tag',
   'kindern',
   'und',
   'niemand',
   'besuchte',
   'geheimen',
   'dorf',
   'die',
   'mit',
   'ruhigen',
   'viele',
   'tiere',
   'er',
   'hund',
   'sich',
   'einen',
   'an',
   'entdeckte',
   'spielte',
   'blumen',
   'entschied',
   'kleiner',
   'rannte',
   'namens',
   'straßen',
   'schöne',
   'garten',
   'zu',
   'einem',
   'eines',
   'durch',
   'erkunden',
   'in',
   'den',
   'lebte',
   'nachmittag',
   'kannte'],
  'unknowns_tokens': [],
  'bloom_filter': 'ocrqa-wp_v1.0.6-de.bloom'}}

Once you set `diagnostics` to *True* , an additional key, `diagnostics`, will be added to the dictionary. The value of this key contains all known and unknown tokens, as well as the name of the Bloom filter used. In this example, we can see that there are no unknown words, meaning every word exists in this specific Bloom filter.

**Example 4**: `model_id`

In [22]:
# Using the same German text example as before
ocrqa_pipeline(de_text, model_id=True)

{'language': 'de', 'score': 1.0, 'bloom_filter': 'ocrqa-wp_v1.0.6-de.bloom'}

Similar to the `diagnostics` attribute, the `model_id` attribute is a simpler version. If set to `True`, the pipeline will return an additional key, `bloom_filter`, with the value indicating the Bloom filter that was used for the analysis.

**Example 5**: `supported languages`

In [23]:
# Using the same German text example as before
ocrqa_pipeline(de_text, supported_languages=True)

{'language': 'de',
 'score': 1.0,
 'supported_languages': ['fr', 'lb', 'de', 'en']}

Once `supported_languages` is set to *True*, the pipeline returns an additional key, `supported_languages`, with a value containing a list of all currently supported languages (i.e., languages that have a corresponding Bloom filter).

**Example 6**: All at once

In [24]:
# Using the same German text example as before
ocrqa_pipeline(de_text, language="fr", version="1.0.5", diagnostics=True, model_id=True, supported_languages=True)

{'language': 'fr',
 'score': 0.8,
 'diagnostics': {'known_tokens': ['von',
   'jeden',
   'diesem',
   'tages',
   'max',
   'ein',
   'fand',
   'tag',
   'kindern',
   'und',
   'niemand',
   'geheimen',
   'dorf',
   'die',
   'mit',
   'viele',
   'tiere',
   'er',
   'hund',
   'sich',
   'einen',
   'an',
   'blumen',
   'kleiner',
   'namens',
   'straßen',
   'schöne',
   'garten',
   'zu',
   'einem',
   'eines',
   'durch',
   'in',
   'den',
   'nachmittag',
   'kannte'],
  'unknowns_tokens': ['entdeckte',
   'spielte',
   'besuchte',
   'erkunden',
   'entschied',
   'rannte',
   'ruhigen',
   'lebte'],
  'bloom_filter': 'ocrqa-wp_v1.0.5-fr.bloom'},
 'supported_languages': ['fr', 'lb', 'de', 'en']}

You can use a mix of additional parameters, or all of them at once, to gain a deeper understanding of your QA score. In the example above, we set the language to French, which results in many unknown tokens being identified, as the Bloom filter used may not cover certain French words.

---
### Edge cases

In [25]:
short_de_text_with_unusual_name = "Glebs geht ins Büro."

In [26]:
# Example 1: Very short sentence
ocrqa_pipeline(short_de_text_with_unusual_name, diagnostics=True)

{'language': 'lb',
 'score': 0.8,
 'diagnostics': {'known_tokens': ['büro', 'ins', 'geht'],
  'unknowns_tokens': ['glebs'],
  'bloom_filter': 'ocrqa-wp_v1.0.5-lb.bloom'}}